In [18]:
# Import modules
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris


# Import PySwarms
import pyswarms as ps

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
n_inputs = 4
n_hidden = 20
n_classes = 3

num_samples = 112

In [20]:
def logits_function(p, X):
    # Roll-back the weights and biases
    W1 = p[0:80].reshape((n_inputs,n_hidden))
    b1 = p[80:100].reshape((n_hidden,))
    W2 = p[100:160].reshape((n_hidden,n_classes))
    b2 = p[160:163].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    logits = a1.dot(W2) + b2 # Pre-activation in Layer 2
    return logits          # Logits for Layer 2

In [21]:
# Forward propagation
def forward_prop(params, X,y):
    logits = logits_function(params, X)

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood

    corect_logprobs = -np.log(probs[range(num_samples), y])
    loss = np.sum(corect_logprobs) / num_samples

    return loss

In [22]:
def Get_PSO(f):
    # Initialize swarm
    options = {'c1': 2.05, 'c2': 2.05, 'w': 2.05 * 0.729}

    # Call instance of PSO
    dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
    optimizer = ps.single.GlobalBestPSO(n_particles=20, dimensions=dimensions, options=options)

    # Perform optimization
    cost, pos = optimizer.optimize(f, iters=1000)
    return pos

In [23]:
def predict(pos, X):
    logits = logits_function(pos, X)
    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [24]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split

labelencoder = LabelEncoder()

def Start_Iris():
    data = pd.read_csv('databases/iris.data', names = ['sepal length', 'sepal width', 'petal length', 'petal width', 'class'])
    target = labelencoder.fit_transform(data['class'].values)
    data = data.drop('class', axis = 1).values
    x_train, x_test, y_train, y_test = train_test_split(data, target)

    def f(x):
        n_particles = x.shape[0]
        j = [forward_prop(x[i], x_train, y_train) for i in range(n_particles)]
        return np.array(j)

    pos = Get_PSO(f)
    scores = (predict(pos, x_test) == y_test)

    print("\n Iris \n")
    print("Mean: ", scores.mean())
    print("Standard Deviation: ", scores.std())


In [25]:
Start_Iris()

2022-03-21 08:08:24,791 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|1000/1000, best_cost=2.11e-13
2022-03-21 08:08:38,744 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 2.1138547261874334e-13, best pos: [ 1.15845102e+00  1.19011022e-01 -1.42770492e+00  9.67014447e-02
  1.64960449e+00  9.76479872e-01  4.23261772e+00  8.84748721e-01
  1.54034374e+00  4.75452477e-01 -3.17933614e+00 -7.19183701e+00
 -1.37851314e+00  1.10329063e+00 -2.64800403e+00  5.47366858e+00
 -7.22497463e+00  7.10024579e-01 -5.04866470e-01  3.65825445e+00
  1.98005788e+00 -1.06277762e+00 -9.54339852e-01  1.27701909e+00
  1.67922063e-01  9.09030683e-01  1.32794051e+00  1.20919244e+00
  6.26334705e-01 -1.86844983e+00 -9.75902221e+00  5.39861844e-01
  1.52988555e+00 -1.38178418e+00  1.55710456e-01 -1.16108476e+00
 -1.38778222e+00 -1.31308056e+00  1.58953219e+00  2.27327585e-01
  5.86983583e-01 -


 Iris 

Mean:  0.9736842105263158
Standard Deviation:  0.16007269816574266
